<a href="https://colab.research.google.com/github/viksimenaite/NaturalLanguageModel/blob/main/%E2%80%9E2_GMM_Lab_darbas%E2%80%9C_kopija.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Roberta model

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 26.7MB/s 
     |████████████████████████████████| 1.2MB 47.6MB/s 
     |████████████████████████████████| 8.2MB 45.1MB/s 
     |████████████████████████████████| 235kB 44.0MB/s 
     |████████████████████████████████| 122kB 53.6MB/s 
     |████████████████████████████████| 51kB 9.6MB/s 
     |████████████████████████████████| 2.3MB 42.5MB/s 
     |████████████████████████████████| 1.8MB 44.7MB/s 
     |████████████████████████████████| 3.3MB 47.9MB/s 
     |████████████████████████████████| 81kB 368kB/s 
     |████████████████████████████████| 112kB 48.0MB/s 
     |████████████████████████████████| 174kB 50.3MB/s 
     |████████████████████████████████| 4.2MB 52.8MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 112kB 58.2MB/s 
     |████████████████████████████████| 245kB 58.4MB/s 
     |████████████████████████████████| 901kB 40.9MB/s 
     |████████████████████████████████| 102kB 13.8MB/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel
import numpy as np
import json
import math
import torch

# Taking training and evaluation data

In [ ]:
with open('/content/drive/MyDrive/DataSet/train-v2.0.json', 'r') as file:
    training_data_set = json.load(file)

training_data_set = [item for topic in training_data_set['data'] for item in topic['paragraphs'] ]

with open('/content/drive/MyDrive/DataSet/dev-v2.0.json', 'r') as file:
    evaluation_data_set = json.load(file)

evaluation_data_set = [item for topic in evaluation_data_set['data'] for item in topic['paragraphs'] ]

# Divide data sets into smaller chunks

In [ ]:
def divided_data_chunk(data_set, chunk_number):
    n = math.ceil(len(data_set)/chunk_number)

    for x in range(0, len(data_set), n):
        chunk = data_set[x: n+x]

        if len(chunk) < n:
            chunk = chunk + [None for y in range(n-len(chunk))]
        yield chunk

training_chunks = list(divided_data_chunk(training_data_set, chunk_number=1))

eval_chunks = list(divided_data_chunk(evaluation_data_set, 3))

# Model training

In [ ]:
is_cuda_available = torch.cuda.is_available()

training_args = {
    'num_train_epochs': 3,
    'train_batch_size': 25,
}

model = QuestionAnsweringModel('roberta', 'roberta-base', args=training_args, use_cuda=is_cuda_available)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

In [ ]:
model.train_model(training_chunks[0], output_dir='/content/drive/MyDrive/training', show_running_loss=True)

add example index and unique id: 100%|██████████| 130319/130319 [00:00<00:00, 874388.12it/s]


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/5258 [00:01<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 3:   0%|          | 0/5258 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/5258 [00:00<?, ?it/s]

(15774, 0.6995430851462179)

# Evaluating trained model

In [ ]:
trained_model = QuestionAnsweringModel('roberta', '/content/drive/MyDrive/training/outputs', args=training_args, use_cuda=is_cuda_available)

In [ ]:
trained_model.eval_model(eval_chunks[0], output_dir = '/content/drive/MyDrive/evaluating')

add example index and unique id: 100%|██████████| 3784/3784 [00:00<00:00, 873582.47it/s]


Running Evaluation:   0%|          | 0/490 [00:00<?, ?it/s]

({'correct': 1981,
  'eval_loss': -8.085578762755102,
  'incorrect': 156,
  'similar': 1647},
 {'correct_text': {'56ddde6b9a695914005b9629': '10th and 11th centuries',
   '56ddde6b9a695914005b962a': 'Denmark, Iceland and Norway',
   '5ad39d53604f3c001a3fe8d1': '',
   '5ad39d53604f3c001a3fe8d2': '',
   '5ad39d53604f3c001a3fe8d3': '',
   '5ad39d53604f3c001a3fe8d4': '',
   '5ad3a266604f3c001a3fea27': '',
   '5ad3a266604f3c001a3fea28': '',
   '5ad3a266604f3c001a3fea29': '',
   '5ad3a266604f3c001a3fea2b': '',
   '5ad3ab70604f3c001a3feb89': '',
   '5ad3ab70604f3c001a3feb8a': '',
   '56dde0ba66d3e219004dad75': '911',
   '5ad3ad61604f3c001a3fec0d': '',
   '5ad3ad61604f3c001a3fec0e': '',
   '5ad3ad61604f3c001a3fec0f': '',
   '5ad3ad61604f3c001a3fec10': '',
   '5ad3ae14604f3c001a3fec39': '',
   '5ad3ae14604f3c001a3fec3a': '',
   '56dde27d9a695914005b9652': 'north',
   '5ad3af11604f3c001a3fec63': '',
   '5ad3af11604f3c001a3fec64': '',
   '5ad3af11604f3c001a3fec65': '',
   '56dde2fa66d3e219004dad9

# Predicting

One question predicting

In [ ]:
context_text = "Mary got the milk there. John moved to the bedroom. Sandra went back to the kitchen. Mary travelled to the hallway."
question = "Where is Mary ?"

predictions = trained_model.predict(
    [
        {
            "context": context_text,
            "qas": [
                {
                    "question": question,
                    "id": "0",
                }
            ],
        }
    ]
)

print(predictions)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 7653.84it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '0', 'answer': ['Mary travelled to the hallway.', 'hallway.', 'the hallway.', 'travelled to the hallway.', 'to the hallway.', 'Mary got the milk there. John moved to the bedroom. Sandra went back to the kitchen. Mary travelled to the hallway.', 'Sandra went back to the kitchen. Mary travelled to the hallway.', 'John moved to the bedroom. Sandra went back to the kitchen. Mary travelled to the hallway.', '.', '', 'Mary', 'bedroom. Sandra went back to the kitchen. Mary travelled to the hallway.', 'the bedroom. Sandra went back to the kitchen. Mary travelled to the hallway.', 'kitchen. Mary travelled to the hallway.', 'the kitchen. Mary travelled to the hallway.', 'back to the kitchen. Mary travelled to the hallway.', 'Mary travelled', 'moved to the bedroom. Sandra went back to the kitchen. Mary travelled to the hallway.', 'went back to the kitchen. Mary travelled to the hallway.']}], [{'id': '0', 'probability': [0.6779825922201128, 0.15276292921053852, 0.09998889325995573, 0.0524

Multiple questions(bonus)

In [ ]:
path = '/content/drive/MyDrive/'
question_id = 0

with open(path + 'answers_to_bonus.csv', 'w') as answers_file:
  with open(path + 'task_2_test_set_questions.txt', "r") as questions_file:
    for line in questions_file:
      current_line_content = line.split(",")

      predictions = trained_model.predict([
        {
            "context": current_line_content[0],
                "qas": [
                    {
                        "question": current_line_content[1],
                        "id": question_id,
                    }
                ],
        }
      ])
      
      answer = predictions[0][0]['answer'][0]
      answers_file.write(str(answer) + '\n')
      question_id += 1


FileNotFoundError: ignored